In [ ]:
import os
import numpy as np
import random
import scipy.sparse as sparse
from scipy.sparse import *
from sklearn.utils import shuffle
import pandas as pd
from math import ceil
from tqdm import trange
from sklearn.metrics import *
import sys
import pickle
from itertools import islice

In [ ]:
file_dir = './data/Ciao/'
pos_file = open(file_dir+"/positive_feedback_dataframe.pkl",'rb')
soc_file = open(file_dir+"/social_positive_feedback_dataframe.pkl",'rb')
P = positive_df = pickle.load(pos_file)
SP = social_positive_df = pickle.load(soc_file)
print('data dimension: \n', positive_df.shape)

In [ ]:
def create_matrix(data):
    for col in ('item', 'user', 'rating'):
        data[col] = data[col].astype('category')
    
    code_user = dict(zip(data['user'].cat.codes, data['user']))
    user_code = dict(zip(data['user'], data['user'].cat.codes))
    code_item = dict(zip(data['item'].cat.codes, data['item']))
    item_code = dict(zip(data['item'], data['item'].cat.codes))
    
    mappings = {'code_user' : code_user, 'user_code' : user_code, 'code_item' : code_item, 'item_code' : item_code}

    ratings = csr_matrix((data['rating'], (data['user'].cat.codes, data['item'].cat.codes)))
    ratings.eliminate_zeros()
    return ratings, data, mappings

In [ ]:
X, df, mappings = create_matrix(positive_df)

In [ ]:
df.head()

In [ ]:
def create_train_test(ratings, test_size = 0.2, seed = 1234):
    assert test_size < 1.0 and test_size > 0.0

    # Dictionary Of Keys based sparse matrix is more efficient
    # for constructing sparse matrices incrementally compared with csr_matrix
    train = ratings.copy().todok()
    test = dok_matrix(train.shape)
    
    # for all the users assign randomly chosen interactions
    # to the test and assign those interactions to zero in the training;
    # when computing the interactions to go into the test set, 
    # remember to round up the numbers (e.g. a user has 4 ratings, if the
    # test_size is 0.2, then 0.8 ratings will go to test, thus we need to
    # round up to ensure the test set gets at least 1 rating)
    rstate = np.random.RandomState(seed)
    for u in range(ratings.shape[0]):
        split_index = ratings[u].indices
        n_splits = ceil(test_size * split_index.shape[0])
        test_index = rstate.choice(split_index, size = n_splits, replace = False)
        test[u, test_index] = ratings[u, test_index]
        train[u, test_index] = 0
    
    train, test = train.tocsr(), test.tocsr()
    return train, test

In [ ]:
X_train, X_test = create_train_test(X, test_size=0.1, seed=20191004)

In [ ]:
class Sample:
    def __init__(self, P, SP):
        self.P = P
        self.SP = SP
        
    def static(self, social = False):
        user = np.random.choice(self.P['user'])
        
        items = {}
        
        items['P'] = np.random.choice(self.P.query('user == @user')['item'])
        tot_items = set(self.P['item'])
        if social:
            items['SP'] = np.random.choice(self.SP.query('user == @user')['item'])
        
        pos_items = []
        for key in items:
            pos_items.append(items[key])
        
        neg_items = list(tot_items - set(pos_items))
                
        items['N'] = np.random.choice(neg_items)
        
        return user, items

In [ ]:
def get_mappings(code, key):
    return code[key]

In [ ]:
class BPR:

    def __init__(self, learning_rate = 0.01, n_factors = 15, n_iters = 10, 
                 reg_u = 0.01, reg_i = 0.01, seed = 1234, verbose = True):
        self.reg_u = reg_u
        self.reg_i = reg_i
        self.seed = seed
        self.verbose = verbose
        self.n_iters = n_iters
        self.n_factors = n_factors
        self.batch_size = 1
        self.learning_rate = learning_rate
        
        # to avoid re-computation at predict
        self._prediction = None
        
    def fit(self, ratings, sampler):
        n_users, n_items = ratings.shape
        
        # ensure batch size makes sense, since the algorithm involves
        # for each step randomly sample a user, thus the batch size
        # should be smaller than the total number of users or else
        # we would be sampling the user with replacement
        batch_size = self.batch_size
        if n_users < batch_size:
            batch_size = n_users
            sys.stderr.write('WARNING: Batch size is greater than number of users,'
                             'switching to a batch size of {}\n'.format(n_users))

        batch_iters = n_users // batch_size
        
        # initialize random weights
        rstate = np.random.RandomState(self.seed)
        self.user_factors = rstate.normal(size = (n_users, self.n_factors))
        self.item_factors = rstate.normal(size = (n_items, self.n_factors))
        
        # progress bar for training iteration if verbose is turned on
        loop = range(self.n_iters)
        if self.verbose:
            loop = trange(self.n_iters, desc = self.__class__.__name__)
        
        for _ in loop:
            for _ in range(batch_iters):
                user, items = sampler.static()
                sampled_users = np.zeros(self.batch_size, dtype = np.int)
                sampled_users[0] = get_mappings(mappings['user_code'], user)
                
                sampled_pos_items = np.zeros(self.batch_size, dtype = np.int)
                sampled_pos_items[0] = get_mappings(mappings['item_code'], items['P'])
                
                '''
                sampled_soc_pos_items = np.zeros(self.batch_size, dtype = np.int)
                sampled_soc_pos_items[0] = get_mappings(mappings['item_code'], items['SP'])
                '''
                
                sampled_neg_items = np.zeros(self.batch_size, dtype = np.int)
                sampled_neg_items[0] = get_mappings(mappings['item_code'], items['N'])
                
                self._update(sampled_users, sampled_pos_items, sampled_neg_items)

        return self
                
    def _update(self, u, i, j):
        """
        update according to the bootstrapped user u, 
        positive item i and negative item j
        """
        user_u = self.user_factors[u]
        item_i = self.item_factors[i]
        item_j = self.item_factors[j]
        
        # decompose the estimator, compute the difference between
        # the score of the positive items and negative items; a
        # naive implementation might look like the following:
        # r_ui = np.diag(user_u.dot(item_i.T))
        # r_uj = np.diag(user_u.dot(item_j.T))
        # r_uij = r_ui - r_uj
        
        # however, we can do better, so
        # for batch dot product, instead of doing the dot product
        # then only extract the diagonal element (which is the value
        # of that current batch), we perform a hadamard product, 
        # i.e. matrix element-wise product then do a sum along the column will
        # be more efficient since it's less operations
        # http://people.revoledu.com/kardi/tutorial/LinearAlgebra/HadamardProduct.html
        # r_ui = np.sum(user_u * item_i, axis = 1)
        #
        # then we can achieve another speedup by doing the difference
        # on the positive and negative item up front instead of computing
        # r_ui and r_uj separately, these two idea will speed up the operations
        # from 1:14 down to 0.36
        r_uij = np.sum(user_u * (item_i - item_j), axis = 1)
        sigmoid = np.exp(-r_uij) / (1.0 + np.exp(-r_uij))
        
        # repeat the 1 dimension sigmoid n_factors times so
        # the dimension will match when doing the update
        sigmoid_tiled = np.tile(sigmoid, (self.n_factors, 1)).T

        # update using gradient descent
        grad_u = sigmoid_tiled * (item_j - item_i) + self.reg_u * user_u
        grad_i = sigmoid_tiled * -user_u + self.reg_i * item_i
        grad_j = sigmoid_tiled * user_u + self.reg_i * item_j
        self.user_factors[u] -= self.learning_rate * grad_u
        self.item_factors[i] -= self.learning_rate * grad_i
        self.item_factors[j] -= self.learning_rate * grad_j
        return self

    def predict(self):
        """
        Obtain the predicted ratings for every users and items
        by doing a dot product of the learnt user and item vectors.
        The result will be cached to avoid re-computing it every time
        we call predict, thus there will only be an overhead the first
        time we call it. Note, ideally you probably don't need to compute
        this as it returns a dense matrix and may take up huge amounts of
        memory for large datasets
        """
        if self._prediction is None:
            self._prediction = self.user_factors.dot(self.item_factors.T)

        return self._prediction

    def _predict_user(self, user):
        """
        returns the predicted ratings for the specified user,
        this is mainly used in computing evaluation metric
        """
        user_pred = self.user_factors[user].dot(self.item_factors.T)
        return user_pred
    
    def recommend(self, data, N = 5):
        """
        Returns the top N ranked items for given user id,
        excluding the ones that the user already liked
        
        Parameters
        ----------
        ratings : scipy sparse csr_matrix, shape [n_users, n_items]
            sparse matrix of user-item interactions 
        
        N : int, default 5
            top-N similar items' N
        
        Returns
        -------
        recommendation : 2d ndarray, shape [number of users, N]
            each row is the top-N ranked item for each query user
        """
        n_users = data.shape[0]
        recommendation = np.zeros((n_users, N))
        scores = np.zeros((n_users, N))
        users = []
        ranks = []
        for user in range(n_users):
            users.append([user]*N)
            ranks.append([i for i in range(1,N+1)])
            topN_items, topN_scores = self.recommend_user(data, user, N)
            recommendation[user], scores[user] = topN_items, topN_scores

        return recommendation, scores, users, ranks
    
    def get_item_ratings(self, data, u):
        
        if u not in self.item_ratings:
            items = data[u].indices
            ratings = data[u].data
            
            self.item_ratings[u] = []
            for i in range(len(items)):
                self.item_ratings[u].append((items[i], ratings[i]))
                
        return self.item_ratings[u]
        

    def recommend_user(self, data, u, N, validation = True):
        """the top-N ranked items for a given user"""
        scores = self._predict_user(u)

        # compute the top N items, removing the items that the user already liked
        # from the result and ensure that we don't get out of bounds error when 
        # we ask for more recommendations than that are available
        liked = set(data[u].indices)
        count = N + len(liked)
        if count < scores.shape[0]:

            # when trying to obtain the top-N indices from the score,
            # using argpartition to retrieve the top-N indices in 
            # unsorted order and then sort them will be faster than doing
            # straight up argort on the entire score
            # http://stackoverflow.com/questions/42184499/cannot-understand-numpy-argpartition-output
            ids = np.argpartition(scores, -count)[-count:]
            best_ids = np.argsort(scores[ids])[::-1]
            best = ids[best_ids]
            best_scores = scores[best]
        else:
            best = np.argsort(scores)[::-1]
            best_scores = scores[best]    

        topN_items = []
        topN_scores = []
        for i in range(len(best)):
            if best[i] not in liked:
                topN_items.append(best[i])
                topN_scores.append(best_scores[i])
                
        topN_items = list(islice((item for item in topN_items), N))
        topN_scores = list(islice((score for score in topN_scores), N))
        return topN_items, topN_scores

    def get_similar_items(self, N = 10, item_ids = None):
        """
        return the top N similar items for itemid, where
        cosine distance is used as the distance metric
        
        Parameters
        ----------
        N : int, default 5
            top-N similar items' N
            
        item_ids : 1d iterator, e.g. list or numpy array, default None
            the item ids that we wish to find the similar items
            of, the default None will compute the similar items
            for all the items
        
        Returns
        -------
        similar_items : 2d ndarray, shape [number of query item_ids, N]
            each row is the top-N most similar item id for each
            query item id
        """
        # cosine distance is proportional to normalized euclidean distance,
        # thus we normalize the item vectors and use euclidean metric so
        # we can use the more efficient kd-tree for nearest neighbor search;
        # also the item will always to nearest to itself, so we add 1 to 
        # get an additional nearest item and remove itself at the end
        normed_factors = normalize(self.item_factors)
        knn = NearestNeighbors(n_neighbors = N + 1, metric = 'euclidean')
        knn.fit(normed_factors)

        # returns a distance, index tuple,
        # we don't actually need the distance
        if item_ids is not None:
            normed_factors = normed_factors[item_ids]

        _, items = knn.kneighbors(normed_factors)
        similar_items = items[:, 1:].astype(np.uint32)
        return similar_items

In [ ]:
sampler = Sample(positive_df, social_positive_df)

In [ ]:
# parameters were randomly chosen
bpr_params = {'reg_u': 0.015,
              'reg_i': 0.025,
              'learning_rate': 0.01,
              'n_iters': 100,
              'n_factors': 10}

bpr = BPR(**bpr_params)
bpr.fit(X_train, sampler)

In [ ]:
def auc_score(model, ratings):
    """
    computes area under the ROC curve (AUC).
    The full name should probably be mean
    auc score as it is computing the auc
    for every user's prediction and actual
    interaction and taking the average for
    all users
    
    Parameters
    ----------
    model : BPR instance
        Trained BPR model
        
    ratings : scipy sparse csr_matrix, shape [n_users, n_items]
        sparse matrix of user-item interactions
    
    Returns
    -------
    auc : float 0.0 ~ 1.0
    """
    auc = 0.0
    
    n_users, n_items = ratings.shape
    for user, row in enumerate(ratings):
        y_pred = model._predict_user(user)
        y_true = np.zeros(n_items)
        y_true[row.indices] = 1
        auc += roc_auc_score(y_true, y_pred)
    auc /= n_users
    return auc

In [ ]:
print(auc_score(bpr, X_test))

In [ ]:
bpr.get_similar_items(N = 5)

In [ ]:
recommendation, scores, users, ranks = bpr.recommend(X_test, N = 5)

In [ ]:
flatten = lambda l: [item for sublist in l for item in sublist]

In [ ]:
df_test = pd.DataFrame({'user': flatten(users), 'item': flatten(recommendation)})

In [ ]:
df_test['item'] = [get_mappings(mappings['code_item'], item) for item in df_test['item'].astype(int)]
df_test['user'] = [get_mappings(mappings['code_user'], user) for user in df_test['user'].astype(int)]

In [ ]:
df_test.head()

In [ ]:
len(df_test['item'])

In [ ]:
users = []
items = []
ratings = []

In [ ]:
for i in range(X_test.shape[0]):
    y = X_test.getrow(i).indices
    if len(y) > 0:
        data = X_test.getrow(i).data
        user = get_mappings(mappings['code_user'], i)
        item = [get_mappings(mappings['code_item'], y_i) for y_i in y]
        users.extend([user]*len(y))
        items.extend(item)
        ratings.extend(data)

In [ ]:
test_data = pd.DataFrame({'user': users, 'item': items, 'rating': ratings})

In [ ]:
from lenskit import topn

In [ ]:
test_data.head()

In [ ]:
df_test.head()

In [ ]:
ndcg = topn.ndcg(df_test, test_data)

In [ ]:
ndcg

In [ ]:
rla = topn.RecListAnalysis()
rla.add_metric(topn.ndcg)
rla.add_metric(topn.recall)
results = rla.compute(df_test, test_data.drop(columns='rating'))
results.head()

In [ ]:
recall_at_5 = topn.recall(df_test, test_data)

In [ ]:
recall_at_5

In [ ]:
precision_at_5 = topn.precision(df_test, test_data)
precision_at_5

In [ ]:
recommendation2, scores2, users2, ranks2 = bpr.recommend(X_test, N = 50)

In [ ]:
df_test2 = pd.DataFrame({'user': flatten(users2), 'item': flatten(recommendation2)})
df_test2['item'] = [get_mappings(mappings['code_item'], item) for item in df_test2['item'].astype(int)]
df_test2['user'] = [get_mappings(mappings['code_user'], user) for user in df_test2['user'].astype(int)]

In [ ]:
topn.ndcg(df_test2, test_data)

In [ ]:
topn.recall(df_test2,test_data)

In [ ]:
rla = topn.RecListAnalysis()
rla.add_metric(topn.ndcg)
rla.add_metric(topn.recall)
results = rla.compute(df_test2, test_data.drop(columns='rating'))
results.head()